In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年2月2日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

# returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
#     rawDF=rawDF,
#     exp_var_list=exp_var,
#     res_var_list=res_var,
#     model_name_list=model_names_list,
# )
# df = returned_MAPE_table.rename(columns=dict_column_names_JP)
# df = df.set_index("関数名")
# df = df.sort_index(axis=0)
# df = df.sort_index(axis=1)

# df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

# print(
#     df.to_latex(
#         caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
#         label=f"{date}everyModelsMAPEtableIn{benchmark_name.upper()}",
#     )
# )

In [3]:
from sklearn.tree import DecisionTreeRegressor

In [4]:
rawDF

,functionName,functionCallNum,benchmarkName,benchmarkClass,process,intBenchmarkClass,na,nonzer,niter,shift
0,.TAU_application,1.0,cg,A,2,1,14000,11,15,20
1,CG,1.0,cg,A,2,1,14000,11,15,20
2,MPI_Finalize(),1.0,cg,A,2,1,14000,11,15,20
3,INITIALIZE_MPI,1.0,cg,A,2,1,14000,11,15,20
4,MPI_Init(),1.0,cg,A,2,1,14000,11,15,20
...,...,...,...,...,...,...,...,...,...,...
17,MPI_Reduce(),1.0,cg,F,256,65536,54000000,26,100,1500
18,SETUP_SUBMATRIX_INFO,1.0,cg,F,256,65536,54000000,26,100,1500
19,MPI_Comm_size(),1.0,cg,F,256,65536,54000000,26,100,1500
20,MPI_Comm_rank(),1.0,cg,F,256,65536,54000000,26,100,1500


In [5]:
rawDF_CONJ_GRAD = rawDF[rawDF["functionName"] == "CONJ_GRAD"]
rawDF_CONJ_GRAD

,functionName,functionCallNum,benchmarkName,benchmarkClass,process,intBenchmarkClass,na,nonzer,niter,shift
5,CONJ_GRAD,16.0,cg,A,2,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,4,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,8,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,16,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,32,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,64,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,128,1,14000,11,15,20
6,CONJ_GRAD,16.0,cg,A,256,1,14000,11,15,20
2,CONJ_GRAD,31.0,cg,B,2,4,75000,13,75,60
2,CONJ_GRAD,46.0,cg,B,4,4,75000,13,75,60


In [6]:
exp_var

['process', 'na', 'nonzer', 'niter', 'shift']

In [7]:
res_var

['functionCallNum']

In [8]:
rawDF_CONJ_GRAD.shape

(37, 10)

In [9]:
X = rawDF_CONJ_GRAD.filter(items=exp_var)
Y = rawDF_CONJ_GRAD.filter(items=res_var)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1234)

In [11]:
X_train, X_test, Y_train, Y_test

(   process       na  nonzer  niter  shift
 5       32    14000      11     15     20
 2       64  1500000      21    100    500
 2       32   150000      15     75    110
 2        4   150000      15     75    110
 2       64    75000      13     75     60
 2       64   150000      15     75    110
 5        2    14000      11     15     20
 6      256    14000      11     15     20
 5       16    14000      11     15     20
 2        8    75000      13     75     60
 5      128    14000      11     15     20
 5        8    14000      11     15     20
 2       32  9000000      26    100   1500
 2       32  1500000      21    100    500
 2       16  1500000      21    100    500
 2        8   150000      15     75    110
 2       64  9000000      26    100   1500
 2      256  1500000      21    100    500
 2        4  1500000      21    100    500
 5       64    14000      11     15     20
 2        2   150000      15     75    110
 2      128  9000000      26    100   1500
 2      128

In [12]:
reg = DecisionTreeRegressor(max_leaf_nodes = 20)

model = reg.fit(X_train, Y_train)
print(model)

DecisionTreeRegressor(max_leaf_nodes=20)


In [13]:
from sklearn.metrics import r2_score
YHat = model.predict(X_test)
r2 = r2_score(Y_test, YHat)
print("R^2 = ", r2)

R^2 =  0.9516129032258065


In [14]:
from sklearn import tree
from IPython.display import Image
import pydotplus

dot_data = tree.export_graphviz(model, out_file=None, feature_names=rawDF_CONJ_GRAD.feature_names, class_names="functionCallNum", filled=True)
graph = pydotplus.graph_from_dot_data(dot_data)

Image(graph.create_png())

ModuleNotFoundError: No module named 'pydotplus'